In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv('/kaggle/input/test-file/tested.csv', usecols = ['Age', 'Fare', 'Survived'])

In [ ]:
df

In [ ]:
from sklearn.impute import MissingIndicator, SimpleImputer
from sklearn.model_selection import train_test_split

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

from sklearn.compose import ColumnTransformer

In [ ]:
X = df.drop(columns = 'Survived')
y = df['Survived']

In [ ]:
X.isnull().sum()

In [ ]:
X.isnull().mean()*100

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y , test_size=0.2)

### First lets try without Adding missing value columns and check the accuracy

In [ ]:
si = SimpleImputer()

In [ ]:
X_train_imputed = si.fit_transform(X_train)
X_test_imputed = si.transform(X_test)

In [ ]:
lr = LogisticRegression()

In [ ]:
lr.fit(X_train_imputed, y_train)

In [ ]:
y_pred = lr.predict(X_test_imputed)

In [ ]:
accuracy_score(y_test, y_pred)

### Check the accuracy with cross val score

In [ ]:
X_final = si.fit_transform(X)

In [ ]:
from sklearn.model_selection import cross_val_score

In [ ]:
print(np.mean(cross_val_score(lr, X_final, y, cv=10)))

### Now lets try with adding the Missing Value Indicator in the dataset and check the accuracy

In [ ]:
mi = MissingIndicator()

In [ ]:
X_train_ = mi.fit_transform(X_train)

In [ ]:
X_train[['Age_Indicator','Fare_Indicator']] = X_train_

In [ ]:
X_test_ = mi.transform(X_test)

In [ ]:
X_test[['Age_Indicator','Fare_Indicator']] = X_test_

In [ ]:
X_train.head()

In [ ]:
from sklearn.preprocessing import OneHotEncoder

In [ ]:
ohe = OneHotEncoder(sparse=False, drop='first')

In [ ]:
X_train[['Age_encoded','Fare_encoded']] = ohe.fit_transform(X_train[['Age_Indicator', 'Fare_Indicator']] ) 

In [ ]:
X_test[['Age_encoded','Fare_encoded']] = ohe.transform(X_test[['Age_Indicator', 'Fare_Indicator']] ) 

In [ ]:
X_train

In [ ]:
si = SimpleImputer()

In [ ]:
X_train_final = si.fit_transform(X_train[['Age', 'Fare', 'Age_encoded', 'Fare_encoded']])

In [ ]:
X_test_final = si.transform(X_test[['Age', 'Fare', 'Age_encoded', 'Fare_encoded']])

In [ ]:
lr = LogisticRegression()

In [ ]:
lr.fit(X_train_final, y_train)

In [ ]:
y_pred_final = lr.predict(X_test_final)

In [ ]:
accuracy_score(y_test, y_pred_final)

In [ ]:
si_mi = SimpleImputer(add_indicator=True)

In [ ]:
X_si_mi = si_mi.fit_transform(X)

In [ ]:
X_si_mi.shape

In [ ]:
print(np.mean(cross_val_score(lr, X_si_mi, y, cv=10)))